In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.metrics import r2_score
from datetime import datetime, timedelta
import pprint

import getDados
from enums import *
import etl
import graficos

In [ ]:
## Processo de ETL
papel = papeis.BBAS3
intervalo = intervalos.um_dia.value
data_inicio = str(datetime.now().year - 5) + '-01-01'
data_fim = (datetime.now() + timedelta(days=-1)).strftime('%Y-%m-%d')
df = getDados.get_fromYahoo_por_datas(papel=papel, data_inicio=data_inicio, data_fim=data_fim, intervalo=intervalo)
df = etl.padroniza_df_yahoo(df_yahoo=df, papel=papel)
df = etl.add_media_movel_simples(df=df, periodo=9, campo='Fechamento')
df = etl.add_media_movel_exponencial(df=df, periodo=50, campo='Fechamento')
df = etl.desvioPadrao(df=df, periodo=20, campo='Fechamento')
df = etl.bandasBollinger(df=df, campo='MMS11', seCampoNaoExiste=True)
df = etl.ifr(df=df, campo='Fechamento', periodo=14)
df['Max20'] = df['Fechamento'].rolling(window=20).max()
df['Min20'] = df['Fechamento'].rolling(window=20).min()
df['FechamentoSeguinte'] = df.Fechamento.shift(-1)
df_original = df.copy()
df.reset_index(inplace=True)
df.dropna(inplace=True)


## -- -- -- -- -- -- --
## distribuição da base
qtd_linhas = len(df)
qtd_linhas_treino = round(.70 * qtd_linhas)
qtd_linhas_teste = qtd_linhas - qtd_linhas_treino
treino_de = 0
treino_ate = qtd_linhas_treino
teste_de = qtd_linhas_treino
teste_ate = qtd_linhas_treino + qtd_linhas_teste


## -- -- -- -- -- -- --
## separação de features e labels
features = df[['Abertura','Alta','Baixa','Volume','MME50','BandaInferior','Min20','Fechamento']].copy()
labels = df['FechamentoSeguinte'].copy()
X_train = features[:qtd_linhas_treino].copy()
X_test = features[teste_de:teste_ate].copy()
y_train = labels[:qtd_linhas_treino].copy()
y_test = labels[teste_de:teste_ate].copy()


## -- -- -- -- -- -- --
## testa os melhores campos
# features_list = ('Abertura','Alta','Baixa','Volume','MMS9','MME50','DesvioPadrao','BandaSuperior','BandaInferior','IFR','Max20','Min20','Fechamento')
# k_best_features = SelectKBest(k='all')
# k_best_features.fit_transform(features, labels)
# k_best_features_scores = k_best_features.scores_
# raw_pairs = zip(features_list[1:], k_best_features_scores)
# ordered_pairs = list(reversed(sorted(raw_pairs, key=lambda x: x[1])))

# k_best_features_final = dict(ordered_pairs[:15])
# best_features = k_best_features_final.keys()
# print ("Melhores features:")
# print (k_best_features_final)
# df_k_best = pd.DataFrame.from_dict(k_best_features_final, orient='index')
# df_k_best.rename(columns={0: 'Score'}, inplace=True)
# df_k_best.Score = df_k_best.Score.map('{:.2f}'.format)
# df_k_best


## -- -- -- -- -- -- --
## Normalizando os dados de entrada(features)
scaler = MinMaxScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale  = scaler.transform(X_test)


## -- -- -- -- -- -- --
## Treinamento usando regressão linear
lr = linear_model.LinearRegression()
lr.fit(X_train_scale, y_train)
pred= lr.predict(X_test_scale)
cd =r2_score(y_test, pred)


## -- -- -- -- -- -- --
## Usando regressão linear
datas_full = df.Datas[teste_de:teste_ate].copy()
datas_teste = datas_full[teste_de:teste_ate].copy()
fechamento_full = df.Fechamento.copy()
fechamento = fechamento_full[teste_de:teste_ate].copy()

df_previsao = pd.DataFrame({'Datas': datas_full, 'Real': fechamento, 'Previsao': pred})
# df_previsao.Previsao = df_previsao.Previsao.shift(+1)
df_previsao.set_index('Datas', inplace=True)
df_previsao['Diferenca'] =  (df_previsao.Previsao - df_previsao.Real)

a_prever = df_original[['Abertura','Alta','Baixa','Volume','MME50','BandaInferior','Min20','Fechamento']].tail(1)

# -- -- -- -- -- -- --
# Prevendo dados
previsao=scaler.transform( a_prever )
pred=lr.predict(previsao)
desv_pad = df_previsao.Diferenca.rolling(window=20).std()[-2]
print(f'Previsão: {pred[0]:,.2f}, Margem de erro: {desv_pad:,.2f}')

pprint.pprint({
    '1 Qtd': qtd_linhas,
    '2 Qtd Treino': f'{qtd_linhas_treino}_de_{treino_de}_até_{treino_ate}',
    '3 Qtd Teste': f'{qtd_linhas_teste}_de_{teste_de}_até_{teste_ate}',
    'Coeficiente de determinação': f'{cd * 100:.2f}'
}, width=1)

nome_papel = getDados.get_papel(papel._name_)
graficos.LinhasPrevisão(df_previsao=df_previsao.tail(50), papel=nome_papel['nome'])

In [ ]:
hoje = pd.DataFrame({'Datas': [datetime.now().strftime('%Y-%m-%d')], 'Previsao': [pred[0]]})
hoje.set_index('Datas', inplace=True)
df_analise = df_original[['Fechamento']].tail(31).join(df_previsao[['Previsao']].tail(31))
df_analise.rename(columns={'Fechamento': 'Real'}, inplace=True)
df_analise.Previsao = df_analise.Previsao.shift(+1)
df_analise['Diferenca'] =  (df_analise.Previsao - df_analise.Real)
df_analise = pd.concat([df_analise, hoje])
# df_analise.rename(columns={'Fechamento': 'Real'}, inplace=True)
# df_analise['Diferenca'] =  (df_analise.Previsao - df_analise.Real)
# df_analise = pd.concat([df_analise.tail(1), pd.DataFrame({"Previsao": [pred[0]]}) ])
# # graficos.LinhasPrevisão2(df_previsao=df_previsao.tail(50), papel=nome_papel['nome'], df_prever=df_analise)
# df_analise.reset_index(inplace=True)
# df_analise.rename(columns={'index': 'Datas'}, inplace=True)
# df_analise.iloc[1, 0] = datetime.now()
# df_analise.set_index('Datas', inplace=True)
# df_analise['Previsao'][-1] = pred[0]
# df_analise
df_analise['DesvPad'] = df_analise.Diferenca.rolling(window=20).std().map('{:,.2f}'.format)
df_analise.DesvPad = df_analise.DesvPad.shift(+1)
df_analise.Real = df_analise.Real.map('{:,.2f}'.format)
df_analise.Previsao = df_analise.Previsao.map('{:,.2f}'.format)
df_analise.Diferenca = df_analise.Diferenca.map('{:,.2f}'.format)
# df_analise.tail(5)
graficos.LinhasPrevisão2(df_previsao=df_previsao.tail(50), papel=nome_papel['nome'], df_prever=df_analise.tail(2))

In [ ]:
a_prever['Previsao'] = pred[0]
a_prever.rename(columns={'Fechamento': 'Real'}, inplace=True)
a_prever['Diferenca'] =  (a_prever.Previsao - a_prever.Real)
a_prever[['Real', 'Previsao']]
df_prever = pd.concat([ df_previsao[['Real', 'Previsao']].tail(1), a_prever[['Real', 'Previsao', 'Diferenca']] ])
graficos.LinhasPrevisão2(df_previsao=df_previsao.tail(50), papel=nome_papel['nome'], df_prever=df_prever)